In [ ]:
import numpy as np
from plotly import graph_objs as go

import dsplib
import plotlib

In [ ]:
fs_hz = 48e3
signal_duration_sec = 0.2

### The tanh(x) function

In [ ]:
x = np.linspace(-3, 3)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.tanh(x), name='tanh'))
fig.update_layout(xaxis_title='Input magnitude', yaxis_title='Output magnitude')
fig.show()

We will do the analysis for a sin wave, as it makes it easy to interpret frequency domain data

### Frequency impact

In [ ]:
mag = 2
t = dsplib.generate_time_signal(signal_duration_sec=signal_duration_sec, fs_hz=fs_hz)

fig = go.Figure()
freq_points_hz = np.arange(300, fs_hz/2, 200)
labels = []
for tone_freq_hz in freq_points_hz:
    sig = mag * np.sin(2*np.pi * tone_freq_hz * t)
    f, S_in = dsplib.calc_spectrum(sig, fs=fs_hz)
    _, S_out = dsplib.calc_spectrum(np.tanh(sig), fs=fs_hz)

    fig.add_trace(go.Scatter(x=f, y=S_in, visible=False, name="input"))
    fig.add_trace(go.Scatter(x=f, y=S_out, visible=False, line=dict(width=2, dash='dash'), name="tanh"))
    labels.append(int(tone_freq_hz))

fig = plotlib.apply_sliders(fig, lines_num=2, labels=labels, prefix="Frequency: ", suffix=" Hz")

fig.update_layout(
    title=f"Signal spectrum before and after tanh",
    xaxis_title='Frequency, Hz',
    yaxis_title='Spectral density'
)
fig.show()

We can see that as the tone frequency increases, the generated by the `tanh` harmonics end up behind the `fs/2` frequency which leads to aliasing.

### Magnitude impact

In [ ]:
tone_freq_hz = 900
t = dsplib.generate_time_signal(signal_duration_sec=signal_duration_sec, fs_hz=fs_hz, history_smp_num=0)
sig = np.sin(2*np.pi * tone_freq_hz * t)
f, S_in_0 = dsplib.calc_spectrum(sig, fs=fs_hz)

fig = go.Figure()
mag_points = np.arange(0.1, 5, 0.1)
labels = []
for mag in mag_points:
    S_in = S_in_0 + 20*np.log10(mag)
    _, S_out = dsplib.calc_spectrum(np.tanh(mag*sig), fs=fs_hz)

    fig.add_trace(go.Scatter(x=f, y=S_in, visible=False, name="input"))
    fig.add_trace(go.Scatter(x=f, y=S_out, visible=False, line=dict(width=2, dash='dash'), name="tanh"))
    labels.append(f"{mag:.1f}")

fig = plotlib.apply_sliders(fig, lines_num=2, labels=labels, prefix="Magnitude: ", suffix="")

fig.update_layout(
    title=f"Signal spectrum before and after tanh",
    xaxis_title='Frequency, Hz',
    yaxis_title='Spectral density'
)

fig.show()